In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
import pandas as pd
# Load Fear/Greed Index dataset
fg = pd.read_csv('../data/raw/fear_greed_index.csv')

# Load Trader Data dataset
trades = pd.read_csv('../data/raw/historical_data.csv')

In [11]:
fg['date'] = pd.to_datetime(fg['date'])


In [14]:
trades['Timestamp IST'] = pd.to_datetime(trades['Timestamp IST'], format='%d-%m-%Y %H:%M')



In [20]:
trades['Side'] = trades['Side'].str.lower()
trades['Direction'] = trades['Direction'].str.lower()


**Data Inspection Finding:**

- The `Side` column indicates whether the action was a "buy" or "sell".
- The `Direction` column provides much richer detail about each trade, including actions like "open long", "close short", "liquidations", "spot dust conversion", and more.
- These two columns are **NOT always the same** and together describe the full nature of each trade.
- **Both columns are needed** for accurate analysis of trader actions and position types.


In [23]:
print(trades.describe())




       Execution Price   Size Tokens      Size USD  \
count    211224.000000  2.112240e+05  2.112240e+05   
mean      11414.723350  4.623365e+03  5.639451e+03   
min           0.000005  8.740000e-07  0.000000e+00   
25%           4.854700  2.940000e+00  1.937900e+02   
50%          18.280000  3.200000e+01  5.970450e+02   
75%         101.580000  1.879025e+02  2.058960e+03   
max      109004.000000  1.582244e+07  3.921431e+06   
std       29447.654868  1.042729e+05  3.657514e+04   

                       Timestamp IST  Start Position     Closed PnL  \
count                         211224    2.112240e+05  211224.000000   
mean   2025-01-31 12:04:22.915009792   -2.994625e+04      48.749001   
min              2023-05-01 01:06:00   -1.433463e+07 -117990.104100   
25%              2024-12-31 21:00:45   -3.762311e+02       0.000000   
50%              2025-02-24 18:55:00    8.472793e+01       0.000000   
75%              2025-04-02 18:22:00    9.337278e+03       5.792797   
max             

## Data Import & Initial Inspection

- Imported trader dataset (`trades`) and sentiment dataset (`fg`).
- Inspected column names, data types, and previewed a few rows.
- Converted necessary columns to numeric and datetime types.

## Data Cleaning

- Checked for missing values; none found.
- Inspected distributions of key numeric columns to check for outliers.
- Explored 'Side' and 'Direction' columns, and determined both are needed for analysis.

*Note: Cells used for each step were cleaned up for clarity; this summary documents the actions performed so far.*


In [25]:
low_price_count = trades[trades['Execution Price'] < 0.01].shape[0]
high_price_count = trades[trades['Execution Price'] > 50000].shape[0]

print(f"Number of trades with execution price < $0.01: {low_price_count}")
print(f"Number of trades with execution price > $50,000: {high_price_count}")


Number of trades with execution price < $0.01: 1795
Number of trades with execution price > $50,000: 25721


Outlier Investigation: Execution Price

Step 1: Checked for trades with extreme execution prices:

Number of trades with execution price < $0.01: 1,795

Number of trades with execution price > $50,000: 25,721

Step 2: Previewed a sample of both low- and high-price trades. Found that both groups included a variety of coins/assets and overall looked plausible for a diverse trading environment.

Step 3: Since both high and low prices occur frequently and appear consistent with legitimate system activity, I decided to retain all trades in this price range. No removal or modification required at this stage.

In [27]:
zero_usd_count = trades[trades['Size USD'] == 0].shape[0]
large_usd_count = trades[trades['Size USD'] > 1_000_000].shape[0]

print(f"Number of trades with USD size $0: {zero_usd_count}")
print(f"Number of trades with USD size > $1,000,000: {large_usd_count}")

Number of trades with USD size $0: 43
Number of trades with USD size > $1,000,000: 34


In [28]:
# Preview the zero-value trades
display(trades[trades['Size USD'] == 0].head(10))

# Preview the very large-value trades
display(trades[trades['Size USD'] > 1_000_000].head(10))

,Account,Coin,Execution Price,Size Tokens,Size USD,Side,Timestamp IST,Start Position,Direction,Closed PnL,Transaction Hash,Order ID,Crossed,Fee,Trade ID,Timestamp
22820,0x4f93fead39b70a1824f981a54d4e55b278e9f760,@17,0.000131,31.434320,0.0,sell,2024-07-24 05:30:00,31.434320,spot dust conversion,0.0,0x171bc45c0156e35b3270040e00a3e9014600ba2a302b...,30932195268,True,0.0,0.0,1.720000e+12
22824,0x4f93fead39b70a1824f981a54d4e55b278e9f760,@24,0.000158,20.268280,0.0,sell,2024-07-24 05:30:00,20.268280,spot dust conversion,0.0,0x171bc45c0156e35b3270040e00a3e9014600ba2a302b...,30932195280,True,0.0,0.0,1.720000e+12
22827,0x4f93fead39b70a1824f981a54d4e55b278e9f760,@31,0.000021,201.954140,0.0,sell,2024-07-24 05:30:00,201.954140,spot dust conversion,0.0,0x171bc45c0156e35b3270040e00a3e9014600ba2a302b...,30932195285,True,0.0,0.0,1.720000e+12
22829,0x4f93fead39b70a1824f981a54d4e55b278e9f760,@34,0.003047,0.202573,0.0,sell,2024-07-24 05:30:00,0.202573,spot dust conversion,0.0,0x171bc45c0156e35b3270040e00a3e9014600ba2a302b...,30932195288,True,0.0,0.0,1.720000e+12
22831,0x4f93fead39b70a1824f981a54d4e55b278e9f760,@37,0.000040,120.896630,0.0,sell,2024-07-24 05:30:00,120.896630,spot dust conversion,0.0,0x171bc45c0156e35b3270040e00a3e9014600ba2a302b...,30932195290,True,0.0,0.0,1.720000e+12
22961,0x4f93fead39b70a1824f981a54d4e55b278e9f760,@44,0.004080,1.002730,0.0,sell,2024-07-27 05:30:00,1.002730,spot dust conversion,0.0,0xcb9449d0f4460e204cdc040e2d0c9f015100e22c6adc...,31294297549,True,0.0,0.0,1.720000e+12
23161,0x4f93fead39b70a1824f981a54d4e55b278e9f760,@53,0.000402,6.029794,0.0,sell,2024-08-13 05:30:00,6.029794,spot dust conversion,0.0,0x00000000000000000000000000000000000000000000...,34000918786,True,0.0,0.0,1.720000e+12
44168,0xb1231a4a2dd02f2276fa3c5e2a2f3436e6bfed23,@2,0.000052,1.328790,0.0,sell,2024-07-24 05:30:00,1.328790,spot dust conversion,0.0,0x171bc45c0156e35b3270040e00a3e9014600ba2a302b...,30932195253,True,0.0,0.0,1.720000e+12
55284,0x23e7a7f8d14b550961925fbfdaa92f5d195ba5bd,PURR/USDC,0.163590,0.000110,0.0,sell,2025-02-02 05:30:00,0.000110,spot dust conversion,0.0,0x00000000000000000000000000000000000000000000...,68346136586,True,0.0,0.0,1.740000e+12
83840,0x3998f134d6aaa2b6a5f723806d00fd2bbbbce891,@10,0.000286,0.968033,0.0,sell,2024-08-07 05:30:00,0.968033,spot dust conversion,0.0,0x00000000000000000000000000000000000000000000...,33072076127,True,0.0,0.0,1.720000e+12


,Account,Coin,Execution Price,Size Tokens,Size USD,Side,Timestamp IST,Start Position,Direction,Closed PnL,Transaction Hash,Order ID,Crossed,Fee,Trade ID,Timestamp
3997,0x513b8629fe877bb581bf244e326a047b249c4ff1,BTC,79117.0,15.00000,1186755.00,sell,2025-04-10 21:37:00,18.27190,close long,4200.000000,0xaedc38f163839a9a031c0421488d2f02039f00b724c6...,85624889672,False,113.928480,1.080000e+15,1.740000e+12
4231,0x513b8629fe877bb581bf244e326a047b249c4ff1,BTC,84300.0,12.74771,1074631.95,sell,2025-04-12 02:53:00,20.03608,close long,5601.343774,0x5696663620893561ac6804215c83730202250016e6fd...,85903965511,False,51.582333,7.720000e+14,1.740000e+12
4402,0x513b8629fe877bb581bf244e326a047b249c4ff1,BTC,83203.0,42.18301,3509752.98,buy,2025-04-12 04:57:00,-42.18302,close short,2147.115209,0x8dab89d7f31e8aa8aeb404215de75a0201030026282d...,85920870456,False,168.468143,2.920000e+14,1.740000e+12
5429,0x513b8629fe877bb581bf244e326a047b249c4ff1,BTC,84590.0,26.81659,2268415.35,sell,2025-04-12 19:37:00,59.32329,close long,19530.522500,0xc0babe6a642940df3371042167c10b02039500e67b8d...,86030750683,False,0.000000,1.010000e+15,1.740000e+12
5431,0x513b8629fe877bb581bf244e326a047b249c4ff1,BTC,84590.0,13.81216,1168370.61,sell,2025-04-12 19:37:00,32.03871,close long,10059.396130,0x092cc27421f8ae08db17042167c13701e0000a666e7f...,86030750683,False,0.000000,9.280000e+14,1.740000e+12
6381,0x513b8629fe877bb581bf244e326a047b249c4ff1,BTC,85644.0,19.06657,1632937.32,sell,2025-04-15 15:04:00,-0.01800,open short,0.000000,0x6bdb51fc150c36eb3867042195960a0201df00822e18...,86685611326,False,0.000000,3.150000e+14,1.740000e+12
6579,0x513b8629fe877bb581bf244e326a047b249c4ff1,BTC,85580.0,16.24238,1390022.88,buy,2025-04-15 16:09:00,-39.00280,close short,1039.512320,0xc3d3e0b2af608cf3d7900421965218018e00732a6f63...,86695577373,True,347.505720,5.150000e+14,1.740000e+12
7279,0x513b8629fe877bb581bf244e326a047b249c4ff1,BTC,85100.0,17.62552,1499931.75,sell,2025-04-16 22:31:00,52.72340,close long,6034.978048,0x9c0be29cce62f4e151990421aaea1f0205380002f455...,86983564107,False,0.000000,8.360000e+13,1.740000e+12
7347,0x513b8629fe877bb581bf244e326a047b249c4ff1,BTC,85100.0,25.07221,2133645.07,sell,2025-04-16 22:31:00,25.14187,close long,8584.724704,0xa19c7e75d09eb1a32a710421aaea2b02062400fc5b71...,86983564107,False,0.000000,6.030000e+14,1.740000e+12
7357,0x513b8629fe877bb581bf244e326a047b249c4ff1,BTC,85134.0,21.01104,1788753.88,buy,2025-04-16 22:39:00,0.58384,open long,0.000000,0xc9e8524dcd2a5f9543a10421ab034502039c00a4c524...,86985565695,False,0.000000,6.170000e+14,1.740000e+12


Outlier Investigation: Trade Size (USD)
Step 1: Checked for trades with unusual USD sizes:

Number of trades with Size USD = $0: 43

Number of trades with Size USD > $1,000,000: 34

Step 2: Previewed several rows from both groups:

$0 trades were all classified as "spot dust conversion", had zero fees and PnL, repeated timestamps, and sometimes a placeholder transaction hash (all zeros). These match expected patterns for dust conversion events on exchanges.

Million-dollar trades appeared as regular transactions, with plausible prices, coin types, realistic transaction hashes, and varied timestamps. Nothing indicated they were errors.

Step 3: Based on both the counts and the content reviewed, I decided to retain both sets of trades in the dataset. These records are consistent with legitimate trading activity and expected special events.

In [29]:
negative_fee_count = trades[trades['Fee'] < 0].shape[0]
very_high_fee_count = trades[trades['Fee'] > 100].shape[0]

print(f"Number of negative fees: {negative_fee_count}")
print(f"Number of fees > $100: {very_high_fee_count}")

display(trades[trades['Fee'] < 0].head(10))
display(trades[trades['Fee'] > 100].head(10))


Number of negative fees: 2476
Number of fees > $100: 127


,Account,Coin,Execution Price,Size Tokens,Size USD,Side,Timestamp IST,Start Position,Direction,Closed PnL,Transaction Hash,Order ID,Crossed,Fee,Trade ID,Timestamp
28806,0x75f7eeb85dc639d5e99c78f95393aa9a5f1170d4,RUNE,4.28680,1500.0,6430.20,sell,2024-01-22 03:48:00,0.0,open short,0.000000,0xdcfc6c7955a7f5db6ca90407427222015e0040e45047...,7688004466,False,-0.128604,8.290000e+14,1.710000e+12
28807,0x75f7eeb85dc639d5e99c78f95393aa9a5f1170d4,ONDO,0.27205,14139.0,3846.51,buy,2024-01-22 07:28:00,0.0,open long,0.000000,0xcce92ffcbf63ca530b42040743082e01d4005d5aa790...,7700952493,False,-0.076930,4.030000e+14,1.710000e+12
28808,0x75f7eeb85dc639d5e99c78f95393aa9a5f1170d4,ONDO,0.27205,249.0,67.74,buy,2024-01-22 07:28:00,14139.0,open long,0.000000,0xcce92ffcbf63ca530b42040743082e01d4005d5aa790...,7700952493,False,-0.001354,1.070000e+15,1.710000e+12
28809,0x75f7eeb85dc639d5e99c78f95393aa9a5f1170d4,ONDO,0.26833,20000.0,5366.60,buy,2024-01-22 07:28:00,14388.0,open long,0.000000,0xdcacf1355584753131ab040743083d015500120b4de6...,7700958287,False,-0.107332,4.990000e+14,1.710000e+12
28810,0x75f7eeb85dc639d5e99c78f95393aa9a5f1170d4,ONDO,0.27107,100.0,27.11,buy,2024-01-22 07:29:00,34388.0,open long,0.000000,0x94fe00eac90ed43e38c204074308a1013f00fa12168e...,7700984519,False,-0.000542,1.040000e+15,1.710000e+12
28815,0x75f7eeb85dc639d5e99c78f95393aa9a5f1170d4,ONDO,0.28210,25718.0,7255.05,sell,2024-01-22 07:32:00,25798.0,close long,314.042498,0x5cb9bfc4451880f7880d0407430afa01090079027a9c...,7701258291,False,-0.145100,2.100000e+14,1.710000e+12
28822,0x75f7eeb85dc639d5e99c78f95393aa9a5f1170d4,ONDO,0.29514,510.0,150.52,sell,2024-01-22 22:33:00,0.0,open short,0.000000,0x3420bb35c996006181160407456979016a009003fa16...,7765223298,False,-0.003010,7.510000e+14,1.710000e+12
28823,0x75f7eeb85dc639d5e99c78f95393aa9a5f1170d4,ONDO,0.29514,100.0,29.51,sell,2024-01-22 22:33:00,-510.0,open short,0.000000,0x7d6ce7cedc1c745649c6040745697a017d0028635293...,7765223298,False,-0.000590,5.770000e+14,1.710000e+12
28824,0x75f7eeb85dc639d5e99c78f95393aa9a5f1170d4,ONDO,0.29514,1384.0,408.47,sell,2024-01-22 22:33:00,-610.0,open short,0.000000,0xff3984e1fbb5d7418f35040745697b010d00c3d9cb5d...,7765223298,False,-0.008169,2.820000e+14,1.710000e+12
28825,0x75f7eeb85dc639d5e99c78f95393aa9a5f1170d4,ONDO,0.29514,10000.0,2951.40,sell,2024-01-22 22:33:00,-1994.0,open short,0.000000,0xcf2d4e84a902b8afc97d040745697d0124009c20ce36...,7765223298,False,-0.059028,1.470000e+14,1.710000e+12


,Account,Coin,Execution Price,Size Tokens,Size USD,Side,Timestamp IST,Start Position,Direction,Closed PnL,Transaction Hash,Order ID,Crossed,Fee,Trade ID,Timestamp
3997,0x513b8629fe877bb581bf244e326a047b249c4ff1,BTC,79117.0,15.00000,1186755.00,sell,2025-04-10 21:37:00,18.27190,close long,4200.000000,0xaedc38f163839a9a031c0421488d2f02039f00b724c6...,85624889672,False,113.928480,1.080000e+15,1.740000e+12
4037,0x513b8629fe877bb581bf244e326a047b249c4ff1,BTC,79380.0,5.00082,396965.09,sell,2025-04-10 22:26:00,-10.66044,open short,0.000000,0x9889350601aa36af7d220421491c7e020131006a3633...,85639155180,True,133.380270,4.000000e+14,1.740000e+12
4042,0x513b8629fe877bb581bf244e326a047b249c4ff1,BTC,79378.0,5.22550,414789.74,sell,2025-04-10 22:26:00,-17.36205,open short,0.000000,0x9889350601aa36af7d220421491c7e020131006a3633...,85639155180,True,139.369351,7.650000e+14,1.740000e+12
4402,0x513b8629fe877bb581bf244e326a047b249c4ff1,BTC,83203.0,42.18301,3509752.98,buy,2025-04-12 04:57:00,-42.18302,close short,2147.115209,0x8dab89d7f31e8aa8aeb404215de75a0201030026282d...,85920870456,False,168.468143,2.920000e+14,1.740000e+12
4608,0x513b8629fe877bb581bf244e326a047b249c4ff1,BTC,83156.0,7.44264,618900.17,sell,2025-04-12 04:58:00,-26.51459,open short,0.000000,0x6b313e029807198fba0f04215de94801ac001092edd8...,85920946401,True,185.670051,8.530000e+14,1.740000e+12
4620,0x513b8629fe877bb581bf244e326a047b249c4ff1,BTC,83154.0,9.92352,825180.38,sell,2025-04-12 04:58:00,-38.65232,open short,0.000000,0x6b313e029807198fba0f04215de94801ac001092edd8...,85920946401,True,247.554114,8.910000e+14,1.740000e+12
4831,0x513b8629fe877bb581bf244e326a047b249c4ff1,BTC,83335.0,5.52470,460400.87,sell,2025-04-12 05:56:00,-85.24110,open short,0.000000,0xc3bed1c7afb8a40193a504215e8e0402022600c444aa...,85927128991,True,115.100218,9.600000e+14,1.740000e+12
4926,0x513b8629fe877bb581bf244e326a047b249c4ff1,BTC,83292.0,9.92352,826549.83,buy,2025-04-12 05:59:00,-43.67672,close short,-550.755360,0x183a84cbd2555bb4e7e404215e968201680049a3c71f...,85927531011,True,206.637456,5.600000e+14,1.740000e+12
4935,0x513b8629fe877bb581bf244e326a047b249c4ff1,BTC,83294.0,6.46607,538584.83,buy,2025-04-12 05:59:00,-27.21585,close short,-371.799025,0x183a84cbd2555bb4e7e404215e968201680049a3c71f...,85927531011,True,134.646208,2.080000e+14,1.740000e+12
4973,0x513b8629fe877bb581bf244e326a047b249c4ff1,BTC,82878.0,5.82886,483084.26,buy,2025-04-12 08:45:00,5.46644,open long,0.000000,0x5124ea8f418a6fd4c8d0042160703e01d7007bedbd1b...,85946303492,True,120.771064,7.990000e+14,1.740000e+12
